In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression

import src.models.mlflow_logging as ml_logging

ml_logging.start_auto_logging("test_logging_local", "sklearn")


# Random model
X = np.linspace(0, 1, 11)
y = X * 2
X = np.array([[x] for x in X])
y = np.array([[i] for i in y])

model = LinearRegression()
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42
)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
mse = float(sum([np.sqrt((i - j) ** 2) for i, j in zip(y_pred, y_test)]))

2022/10/17 16:56:15 WARNING mlflow.utils.autologging_utils: You are using an unsupported version of sklearn. If you encounter errors during autologging, try upgrading / downgrading sklearn to a supported version, or try upgrading MLflow.


An experiment with that name already exists, logging new run into it.


2022/10/17 16:56:16 INFO mlflow.utils.autologging_utils: Created MLflow autologging run with ID '9e77dd5c0e15438d996e52b73713691d', which will track hyperparameters, performance metrics, model artifacts, and lineage information for the current sklearn workflow
2022/10/17 16:56:19 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "c:\Users\Philip\anaconda3\envs\mi4\lib\site-packages\_distutils_hack\__init__.py:30: UserWarning: Setuptools is replacing distutils."
